In [1]:
import atoti as tt
from pathlib import Path

In [2]:
session = tt.Session.start()

In [3]:
data_source = "target/files"

In [6]:
commit_table = session.read_csv(
    f"{data_source}/commits-*.csv",
    table_name="commits",
    data_types={
        "date": tt.type.LOCAL_DATE,
    })

In [10]:
branch_table = session.read_csv(
    f"{data_source}/branches-*.csv",
    table_name="branch",
    keys={"commit", "branch"})

In [11]:
change_table = session.read_csv(
    f"{data_source}/changes-*.csv",
    table_name="changes",
    keys={"commit", "path"})

In [13]:
renaming_table = session.read_csv(
    f"{data_source}/renamings-*.csv",
    table_name="renamings",
    keys={"commit", "before", "after"}
)

In [14]:
session.tables.schema

```mermaid
erDiagram
  "commits" {
    non-null String "commit"
    nullable int "timestamp"
    non-null LocalDate "date"
  }
  "branch" {
    non-null String PK "branch"
    non-null String PK "commit"
  }
  "changes" {
    non-null String PK "commit"
    non-null String "module"
    non-null String "filename"
    non-null String PK "path"
    nullable int "additions"
    nullable int "deletions"
  }
  "renamings" {
    non-null String PK "commit"
    non-null String PK "before"
    non-null String PK "after"
  }
```


In [15]:
change_table.head()

,,module,filename,additions,deletions
commit,path,,,,
acb6d28d2d7dbada2b9b1fdf8606ed6a5e3f9361,ConverterUtils.java,pivot-server/impl/src/main/java/com/activeviam...,pivot-server/impl,19,4
c50d1a3be4dfa7777efcb17d14db06634bbff762,MemoryUtil.java,composer/core/src/main/java/com/activeviam/tec...,composer/core,31,0
dbd9f579bd022930b521ab51b4441935bd6fe0e0,FilesUtil.java,application/patachou/core/src/main/java/com/ac...,application/patachou/core,52,35
04bc2e91c43c3d6a94e9c5aefd7f9a1177837182,DeleteTableOutput.java,atoti/patachou/server/server-base/src/main/jav...,atoti/patachou/server/server-base,2,2
cc459d6c0f2e894d7149dcaca814535ac4bb0b83,Hierarchy.java,ai/core/visualize-this/src/main/java/com/activ...,ai/core/visualize-this,31,0


In [16]:
commit_table.row_count

9985

In [17]:
commit_table.join(branch_table, commit_table["commit"] == branch_table["commit"])

In [18]:
change_table.join(commit_table, change_table["commit"] == commit_table["commit"])

In [19]:
change_table.join(renaming_table, (change_table["commit"] == renaming_table["commit"]) & (change_table["path"] == renaming_table["after"]))

In [20]:
cube = session.create_cube(name="git", fact_table=change_table)

In [21]:
session.url


'http://localhost:35493'